In [0]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h2>Extract MNIST data</h2>
<p style="font-size:20px">You can change the option of one_hot encoding.

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h2>Define hyperparameters</h2>

In [0]:
#learning rate
lr = 0.001
#number of traning steps
num_steps =10000
#number of batch_size
batch_size = 128

#network parameters
n_hidden_1 = 800
n_hidden_2 = 1000
n_hidden_3 = 800
num_input = 784
num_classes = 10

<h2>Define placeholder and Variables</h2>

In [0]:
tf.reset_default_graph()
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

#Layers weight & bias
weights = {
    'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
    'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_2, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b2'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}

<h2>Define neural network</h2>

In [0]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.nn.relu(tf.matmul(x,weights['W1']) + biases['b1'])
    layer_2_out = tf.nn.relu(tf.matmul(layer_1_out,weights['W2']) +biases['b2'])
    layer_3_out = tf.nn.relu(tf.matmul(layer_2_out,weights['W3']) +biases['b3'])
    out = tf.add(tf.matmul(layer_2_out,weights['Wout']),biases['bout'])
    return out

<h2>Define cost function and accuracy</h2>

In [0]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h2>Execute training</h2>

In [0]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
            print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.047
Testing ACcuracy: 0.0721
step 100, Accuracy= 0.727
Testing ACcuracy: 0.8189
step 200, Accuracy= 0.836
Testing ACcuracy: 0.864
step 300, Accuracy= 0.883
Testing ACcuracy: 0.8902
step 400, Accuracy= 0.875
Testing ACcuracy: 0.9013
step 500, Accuracy= 0.922
Testing ACcuracy: 0.9115
step 600, Accuracy= 0.922
Testing ACcuracy: 0.9121
step 700, Accuracy= 0.938
Testing ACcuracy: 0.9171
step 800, Accuracy= 0.914
Testing ACcuracy: 0.9236
step 900, Accuracy= 0.961
Testing ACcuracy: 0.9291
step 1000, Accuracy= 0.930
Testing ACcuracy: 0.9314
step 1100, Accuracy= 0.977
Testing ACcuracy: 0.9343
step 1200, Accuracy= 0.945
Testing ACcuracy: 0.9343
step 1300, Accuracy= 0.969
Testing ACcuracy: 0.9357
step 1400, Accuracy= 0.977
Testing ACcuracy: 0.9411
step 1500, Accuracy= 0.953
Testing ACcuracy: 0.936
step 1600, Accuracy= 1.000
Testing ACcuracy: 0.9385
step 1700, Accuracy= 0.969
Testing ACcuracy: 0.9421
step 1800, Accuracy= 0.961
Testing ACcuracy: 0.9434
step 1900, Accuracy= 0.969

<h2>Your results</h2>

1.change number of training steps:

| training steps | 500|1000  |  5000|10000
|------  |------|
|   accuracy | 0.8694|0.8702|0.8738| 0.8808

Accuracy increases with training steps increases, so we choose training steps = 10000


2.change activation function:

| activation function | none|tanh   |  sigmoid|relu
|------  |------|
|   accuracy | 0.8734|0.7727|0.7909| 0.818

Though without activation function, accuracy is largest, for more appoporiate for future adjustment, we choose relu as activation function.

3.change neural network structure

| layer | (300,100)|(300,400)  |  (500,400)|(500,400,400)|(500,800,500)| (800,1000,800)
|------  |------|
|   accuracy | 0.818|0.9242|0.9279| 0.9582|0.952|0.96

use 3 hidden layer, hidden layeer size is 800,1000,800

4.change learning rate

| learning rate  |0.01|0.005  |  0.001|0.0005|0.0001
|------  |------|
|   accuracy | 0.96|0.963|0.9675| 0.960|0.955

choose learning rate = 0.001



##final parameter configuration:


lr = 0.001

num_steps =10000

batch_size = 128


n_hidden_1 = 800

n_hidden_2 = 1000

n_hidden_3 = 800

activation fuction: relu

##accuracy = 0.9675


Activation function, number of layers, number of neurons cotribute most to improve the performance of model.

However, the changes caused by number of training steps is very small.

possible explaination: the structure of initial neural network is too simple to predict with high accuracy, change the layers and neurons improve the complexity of model and thus improves accuracy.
